In [1]:
import findspark
findspark.init()


import pyspark
import sys
import re
import random

#from numpy import sqrt
from pyspark import SparkConf, SparkContext
sc = SparkContext(appName = "MovieLens")
sc.addPyFile("similaritycos.py")
sc.addPyFile("movielensfcn.py")
import similaritycos
import movielensfcn
from movielensfcn import parseMovies, uniqueFilter, itemitem
from similaritycos import cosine_similarity

In [2]:
numPartitions = 11
movies_data = sc.textFile("/data/movie-ratings/movies.dat")
ratings_data = sc.textFile("/data/movie-ratings/ratings.dat")

movies= movies_data.map(lambda line: re.split(r'::',line)).map(lambda x: (int(x[0]),(x[1],x[2])))

ratings = ratings_data.map(lambda line: re.split(r'::',line)).map(lambda x: (int(x[0]),(int(x[1]),float(x[2])))).repartition(numPartitions)

user_ratings_data= ratings.join(ratings)

uniqueJoin = user_ratings_data.filter(uniqueFilter)

movie_pairs = uniqueJoin.map(itemitem)

movie_pairs_ratings=movie_pairs.groupByKey()

item_item_similarities = movie_pairs_ratings.mapValues(cosine_similarity).cache()

item_item_sorted=item_item_similarities.sortByKey()

#item_item_filtered = item_item_sorted.filter(lambda x: x[1][0] >= 0.8)

item_item_sorted.saveAsTextFile("movie-similar3")

In [3]:
# if (len(sys.argv)> 1):
#     scoreThreshold = 0.97
#     coOccurenceThreshold = 50
#     movieID = int(sys.argv[1])
#     filteredResults = movie_movie_Similarities.filter(lambda((pair,sim)): \
#         (pair[0] == movieID or pair[1] == movieID) \ 
#         and sim[0] > scoreThreshold and sim[1] > coOccurenceThreshold)

ValueError: invalid literal for int() with base 10: '-f'

In [ ]:
# results = filteredResults.map(lambda((pair,sim)): (sim,pair)).sortByKey(ascending = False).take(10)